# Train and deploy an sklearn CoverType model


In [1]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update   `REGION`, and `ARTIFACT_STORE`  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.

In [2]:
!gsutil ls

gs://artifacts.mlops-dev-env.appspot.com/
gs://dataflow-staging-us-central1-881178567352/
gs://hostedkfp-default-36un4wco1q/
gs://hostedkfp-default-w2dc42i8jo/
gs://jk-mlops-dev-sandbox/
gs://mlops-dev-env-staging/
gs://mlops-dev-env.appspot.com/
gs://mlops-dev-env_cloudbuild/
gs://mlops-dev-workspace/
gs://staging.mlops-dev-env.appspot.com/


In [5]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://mlops-dev-workspace/artifact-store'
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)

DATA_ROOT='gs://workshop-datasets/covertype/data_validation'
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.csv')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'evaluation', 'dataset.csv')

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]


## Explore the dataset

In [6]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape)
print(df_validation.shape)

(431009, 13)
(75000, 13)


In [7]:
df_train.head(4).T

,0,1,2,3
Elevation,3034,2827,2854,3188
Aspect,235,26,27,156
Slope,13,16,12,13
Horizontal_Distance_To_Hydrology,655,85,484,175
Vertical_Distance_To_Hydrology,134,13,22,28
Horizontal_Distance_To_Roadways,5799,3511,618,3666
Hillshade_9am,197,211,215,235
Hillshade_Noon,251,204,214,241
Hillshade_3pm,190,125,134,134
Horizontal_Distance_To_Fire_Points,4094,6008,1295,1719


## Develop a training application

### Configure the `sklearn` training pipeline.

The training pipeline preprocesses data by standardizing all numeric features using `sklearn.preprocessing.StandardScaler` and encoding all categorical features using `sklearn.preprocessing.OneHotEncoder`. It uses stochastic gradient descent linear classifier (`SGDClassifier`) for modeling.

In [8]:
numeric_feature_indexes = slice(0, 10)
categorical_feature_indexes = slice(10, 12)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log', tol=1e-3))
])

### Run training.

In [9]:
X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=2000)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  slice(0, 10, None)),
                                                 ('cat',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64

### Calculate the trained model's accuracy.

In [10]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

0.7042133333333334


### Copy the model to GCS

In [11]:
gcs_model_path = '%s/model/' % ARTIFACT_STORE
local_model_path = '/tmp/model.pkl'

In [12]:
with open(local_model_path, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        
!gsutil cp {local_model_path} {gcs_model_path}
!gsutil ls {gcs_model_path}

Copying file:///tmp/model.pkl [Content-Type=application/octet-stream]...
/ [1 files][  6.0 KiB/  6.0 KiB]                                                
Operation completed over 1 objects/6.0 KiB.                                      
gs://mlops-dev-workspace/artifact-store/model/model.pkl


## Deploy the model to AI Platform Prediction

### Create a model resource

In [25]:
model_name = 'covertype_classifier_sklearn'
labels = "task=classifier,domain=forestry"
filter = 'name:{}'.format(model_name)
models = !(gcloud ai-platform models list --filter={filter} --format='value(name)' --quiet)
models = [model for model in models if model[:7] != 'WARNING']

if not models:
    !gcloud ai-platform models create  $model_name \
    --regions=$REGION \
    --labels=$labels
else:
    print("Model: {} already exists.".format(models[0]))

[]
Created ml engine model [projects/mlops-dev-env/models/covertype_classifier_sklearn].


### Create a model version

In [26]:
model_version = 'v1'
filter = 'name:{}'.format(model_version)
versions = !(gcloud ai-platform versions list --model={model_name} --format='value(name)' --filter={filter})
versions = [version for version in versions if version[:7] != 'WARNING']

if not versions:
    !gcloud ai-platform versions create {model_version} \
    --model={model_name} \
    --origin={gcs_model_path} \
    --runtime-version=1.15 \
    --framework=scikit-learn \
    --python-version=3.7
else:
    print("Model version: {} already exists.".format(versions[0]))

Creating version (this might take a few minutes)......done.                    


In [27]:
!(gcloud ai-platform versions list --model={model_name} --format='value(name)' --filter={filter})

v1


## Predict

### Define predict function

In [28]:
service = discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, model_name, model_version)
print("Service name: {}".format(name))

def caip_predict(instances):
    
  request_body={
      'instances': instances}

  response = service.projects().predict(
      name=name,
      body=request_body

  ).execute()

  if 'error' in response:
    raise RuntimeError(response['error'])

  return response

Service name: projects/mlops-dev-env/models/covertype_classifier_sklearn/versions/v1


### Invoke the service

In [29]:
instance_iterator = X_validation.iterrows()

for num_calls in range(5):
    instances = []
    for num_instances in range(3):
        instances.append(list(next(instance_iterator)[1].values))
    response = caip_predict(instances)
    print(response)
    print("***")
    time.sleep(0.25)

{'predictions': [1, 1, 0]}
***
{'predictions': [0, 1, 1]}
***
{'predictions': [0, 2, 1]}
***
{'predictions': [1, 0, 1]}
***
{'predictions': [1, 1, 0]}
***


## 5. BigQuery logging dataset preparation

### 5.1. Create BQ Dataset

In [32]:
from google.cloud import bigquery

BQ_DATASET_NAME = 'data_validation'
BQ_TABLE_NAME = 'covertype_classifier_logs_sklearn'

client = bigquery.Client(PROJECT_ID)
dataset_names = [dataset.dataset_id for dataset in client.list_datasets(PROJECT_ID)]

dataset = bigquery.Dataset("{}.{}".format(PROJECT_ID, BQ_DATASET_NAME))
dataset.location = "US"

if BQ_DATASET_NAME not in dataset_names:
  dataset = client.create_dataset(dataset)
  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

print("BigQuery Dataset is ready.")

BigQuery Dataset is ready.


### 5.2. Create BQ Table


#### Table schema

In [33]:
import json

table_schema_json = [
  {
    "name": "model", 
    "type": "STRING", 
    "mode": "REQUIRED"
   },
   {
     "name":"model_version", 
     "type": "STRING", 
     "mode":"REQUIRED"
  },
  {
    "name":"time", 
    "type": "TIMESTAMP", 
    "mode": "REQUIRED"
  },
  {
    "name":"raw_data", 
    "type": "STRING", 
    "mode": "REQUIRED"
  },
  {
    "name":"raw_prediction", 
    "type": "STRING", 
    "mode": "NULLABLE"
  },
  {
    "name":"groundtruth", 
    "type": "STRING", 
    "mode": "NULLABLE"
  },
]

json.dump(table_schema_json, open('table_schema.json', 'w'))

#### Ceating an ingestion-time partitioned tables

In [34]:
table = bigquery.Table(
    "{}.{}.{}".format(PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME))

table_names = [table.table_id for table in client.list_tables(dataset)]

if BQ_TABLE_NAME in table_names:
  print("Deleteing BQ Table: {} ...".format(BQ_TABLE_NAME))
  client.delete_table(table)

# table = client.create_table(table)
# table.partition_expiration = 60 * 60 * 24 * 7
# print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))


In [35]:
TIME_PARTITION_EXPERIATION = 60 * 60 * 24 * 7 # week

!bq mk --table \
  --project_id={PROJECT_ID} \
  --time_partitioning_type=DAY \
  --time_partitioning_expiration {TIME_PARTITION_EXPERIATION} \
  {PROJECT_ID}:{BQ_DATASET_NAME}.{BQ_TABLE_NAME} \
  'table_schema.json'

Table 'mlops-dev-env:data_validation.covertype_classifier_logs_sklearn' successfully created.


### 5.3. Configre the AI Platform model version to enable request-response logging to BigQuery

In [36]:
sampling_percentage = 1.0
bq_full_table_name = '{}.{}.{}'.format(PROJECT_ID, BQ_DATASET_NAME, BQ_TABLE_NAME)

In [37]:
logging_config = {
    "requestLoggingConfig":{
        "samplingPercentage": sampling_percentage,
        "bigqueryTableName": bq_full_table_name
        }
    }

response = service.projects().models().versions().patch(
    name=name,
    body=logging_config,
    updateMask="requestLoggingConfig"
    ).execute()

response

{'name': 'projects/mlops-dev-env/operations/update_covertype_classifier_sklearn_v1_1589050819119',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.ml.v1.OperationMetadata',
  'createTime': '2020-05-09T19:00:19Z',
  'operationType': 'UPDATE_VERSION',
  'modelName': 'projects/mlops-dev-env/models/covertype_classifier_sklearn',
  'version': {'name': 'projects/mlops-dev-env/models/covertype_classifier_sklearn/versions/v1',
   'deploymentUri': 'gs://mlops-dev-workspace/artifact-store/model/',
   'createTime': '2020-05-09T18:57:46Z',
   'runtimeVersion': '1.15',
   'state': 'READY',
   'etag': 'a69ff4JaCGU=',
   'framework': 'SCIKIT_LEARN',
   'machineType': 'mls1-c1-m2',
   'pythonVersion': '3.7',
   'requestLoggingConfig': {'samplingPercentage': 1,
    'bigqueryTableName': 'mlops-dev-env.data_validation.covertype_classifier_logs_sklearn'}}}}

### 5.4. Test request-response logging

In [38]:
instance_iterator = X_validation.iterrows()

for num_calls in range(1000):
    instances = []
    for num_instances in range(3):
        instances.append(list(next(instance_iterator)[1].values))
    response = caip_predict(instances)

In [39]:
query = '''
  SELECT * FROM 
  `{}.{}` 
  WHERE model_version = '{}'
  ORDER BY time desc
  LIMIT {}
'''.format(BQ_DATASET_NAME, BQ_TABLE_NAME, model_version, 3)

result = pd.io.gbq.read_gbq(
    query, project_id=PROJECT_ID)


ImportError: Missing optional dependency 'pandas_gbq'. pandas-gbq is required to load data from Google BigQuery. See the docs: https://pandas-gbq.readthedocs.io. Use pip or conda to install pandas_gbq.

In [29]:
result['raw_data']

0    {"instances": [[2665, 114, 10, 190, 62, 1634, ...
1    {"instances": [[2451, 315, 10, 382, 234, 1092,...
2    {"instances": [[2744, 23, 10, 120, 11, 1566, 2...
Name: raw_data, dtype: object

In [30]:
raw_data = json.loads(result['raw_data'][0])

In [31]:
raw_data

{'instances': [[2665,
   114,
   10,
   190,
   62,
   1634,
   238,
   230,
   121,
   1181,
   'Commanche',
   'C4704'],
  [2639, 51, 10, 384, 19, 3132, 225, 219, 127, 2919, 'Commanche', 'C4704'],
  [2662, 41, 10, 60, 0, 1440, 221, 219, 132, 1454, 'Commanche', 'C4704']]}

int

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>